In [12]:
import time
import random
import functools
import numpy as np
from causality_functions import jitter, dependent, simulations_data
from causality_functions import compute_regression, compute_correlation
from causality_functions import add_confidence_stats, confidence_graphs
from causality_functions import stats_graphs, check_generation
import collections

import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 60, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 1000, 15, 21
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.25, 0
m3, k3 = 0.5, 0
m4, k4 = 1.5, 0

d1 = 5
d2 = 1
d3 = 1
d4 = 1

errorA = 2 # not using this
errorB = 0.001
errorC = 0.002
errorX = 0.75
errorY = 0.6

mrange1 = 0,2 #slope ranges
mrange2 = 0,2
mrange3 = 0,2
mrange4 = 0,2

erangeA = 0.2,20 #error range
erangeB = 0.2,200 #error range
erangeC = 0.2,200 #error range
erangeX = 0.2,2 #error range
erangeY = 0.2,2 #error range



def linear(n=100, debug=False):
    
    def compute_A(history, generation):
        A, _, _ = history[-1]
        return jitter(A, errorA) # previous A + error 
        
    def compute_B(A, history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[-1 - d1]
        return dependent(A, m1, k1, error=errorB)
        
    def compute_C(B, history, generation):
        if check_generation(generation, d2):
            _, B, _ = history[-1 - d2]
        return dependent(B, m2, k2, error=errorC)
    
    def next_generation(history, generation):
        A_ = compute_A(history, generation)
        B_ = compute_B(A_, history, generation)
        C_ = compute_C(B_, history, generation)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2])+1)
    history.append([A, 0, 0])
    data = []
    for i in range(n+max([d1, d2])):
        A, B, C = next_generation(history, i)
        history.append((A, B, C))
        data.append((A, B, C))

    if debug:
        data = pd.DataFrame(np.array(data))
        data.to_csv("data.csv")
    return np.array(history[-1])


def radiating(n=100):

    def compute_B(history, generation):
        _, B, _ = history[-1]
        return jitter(B, errorB)

    def compute_A(B, history, generation):
        if check_generation(generation, d1):
            _, B, _ = history[-1 - d1]
        return dependent(B, m1, k1, errorA)

    def compute_C(B, history, generation):
        if check_generation(generation, d2):
            _, B, _ = history[-1 - d2]
            C_ = dependent(B, m2, k2, errorC)
        return dependent(B, m2, k2, errorC)
                           
    def next_generation(history, generation):
        B_ = compute_B(history, generation)
        A_ = compute_A(B_, history, generation)
        C_ = compute_C(B_, history, generation)
        return A_, B_, C_
    

    B = np.random.normal(loc=Bm, scale=Bstd) # normal distribution, Bm - mean and Bstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2])+1)
    history.append((0, B, 0))
    for i in range(n+max([d1, d2])):
        history.append(next_generation(history, i))

    return np.array(history[-1])


def convergent(n=100):

    def next_generation(history, generation):
        if check_generation(generation, d1):
            A, _, _ = history[-1 - d1]
        else:
            A, _, _ = history[-1]

        if check_generation(generation, d2):
            _, _, C = history[-1 - d2]
        else:
            _, _, C = history[-1]

        B_ = jitter(m1*A + m2*C + k1, errorB)
        
        A, _, C = history[-1]
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_
    
    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    C = np.random.normal(loc=Cm, scale=Cstd) # normal distribution, Cm - mean and Cstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]) + 1)
    history.append((A, 0, C))

    for i in range(n + max([d1, d2])):
        A, B, C = next_generation(history, i)
        history.append((A, B, C))
        
    return np.array(history[-1])


def common_cause(n=100):

    def compute_A(X, history, generation):
        if check_generation(generation, d1):
            _, _, _, X = history[-1 - d1]
        
        return dependent(X, m1, k1, errorA)

    def compute_B(X, history, generation):
        if check_generation(generation, d2):
            _, _, _, X = history[-1 - d2]
        
        return dependent(X, m2, k2, errorB)

    def compute_C(X, history, generation):
        if check_generation(generation, d3):            
            _, _, _, X = history[-1 - d3]
            
        return dependent(X, m3, k3, errorC)

    def next_generation(history, generation):
        _, _, _, X = history[-1]
        X_ = jitter(X, errorX)
        A_ = compute_A(X_, history, generation)
        B_ = compute_B(X_, history, generation)
        C_ = compute_C(X_, history, generation)

        return A_, B_, C_, X
        
    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3])+1)
    history.append((0, 0, 0, X))
    for i in range(n+max([d1, d2, d3])):
        A, B, C, X = next_generation(history, i)
        history.append((A, B, C, X))
    A, B, C, _ = history[-1]
    return np.array([A,B,C])


def single_difference_cause(n=100):
    def next_generation(history, generation):
        A, _, _, X = history[-1]
        A_ = jitter(A, errorA)
        X_ = jitter(X, errorX)

        # compute B
        if d1 > 0 and generation >= d1:
            A, _, _, _ = history[-1 - d1]
        else:
            A, _, _, _ = history[-1]

        if d2 > 0 and generation >= d2:
            _, _, _, X = history[-1 - d2]
        else:
            _, _, _, X = history[-1]
            
        B_ = jitter(m1*A + m2*X + k1, errorB)

        # compute C
        if d3 > 0 and generation >= d3:
            _, _, _, X = history[-1 - d3]
        else:
            _, _, _, X = history[-1]

        C_ = dependent(X, m2, k3, errorC)

        return A_, B_, C_, X_

    
    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]) + 1)
    history.append((A, 0, 0, X))
    for i in range(n + max(d1, d2, d3)):
        A,B,C,X = next_generation(history, i)
        history.append((A, B, C, X))

    A, B, C, X = history[-1]
    return np.array([A, B, C])


def double_difference_cause(n=100):
 
    def next_generation(history, generation):
        _, _, _, X, Y = history[-1]
        X_ = jitter(X, errorX)
        Y_ = jitter(Y, errorY)

        # compute A
        if d1 and generation >= d1:
            _, _, _, X, _ = history[-1 - d1]
        else:
            _, _, _, X, _ = history[-1]
        A_ = dependent(X, m1, k1, errorA)

        # compute B
        if d2 > 0 and generation >= d2:
            _, _, _, X, _ = history[-1-d2]
        else:
            _, _, _, X, _ = history[-1]
        if d3 > 0 and generation >= d3:
            _, _, _, _, Y = history[-1-d3]
        else:
            _, _, _, _, Y = history[-1]
        B_ = jitter(m2*X+m3*Y+k2, errorB)

        # compute C
        if d4 > 0 and generation >= d4:
            _, _, _, _, Y = history[-1-d4]
        else:
            _, _, _, _, Y = history[-1]
        C_ = dependent(Y, m4, k4, errorC)

        return A_, B_, C_, X_, Y_

   

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    Y = np.random.normal(loc=Ym, scale=Ystd) # normal distribution, Ym - mean and Ystd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3, d4]) +  1)
    history.append((0, 0, 0, X, Y))
                   
    for i in range(n + max([d1, d2, d3, d4])):
        A, B, C, X, Y = next_generation(history, i)
        history.append((A, B, C, X, Y))

    A, B, C, X, Y = history[-1]
    return np.array([A, B, C])

def update_slopes():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
  
    m1 = select(*mrange1)
    m2 = select(*mrange2)
    m3 = select(*mrange3)
    m4 = select(*mrange4)

def update_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global errorA, errorB, errorC, errorX, errorY

    errorA = select(*erangeA)
    errorB = select(*erangeB)
    errorC = select(*erangeC)
    errorX = select(*erangeX)
    errorY = select(*erangeY)
    
def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes()
        update_errors()
        ABC = simulations_data(linear, samplesize) # you can change pathway here
        r = compute_regression(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

#linear(100, True)
d, ABC_all = overall_simulation()

In [13]:
add_confidence_stats(d, ABC_all)
confidence_graphs(d)

alt.VConcatChart(...)

In [14]:
stats_graphs(d)

alt.VConcatChart(...)

In [15]:
A, B, C = random.choice(ABC_all).transpose()
data = pd.DataFrame({"A":A,"B":B,"C":C })
data['D'] = 'X'
brush = alt.selection_interval()
AB = alt.Chart(data).mark_circle().encode(
    x="A:Q",
    y="B:Q",
    color=alt.condition(brush, "D:N",alt.value('lightgray'))
   ).add_params(
    brush)
BC= AB.encode(
    x="B",
    y="C")
AC = AB.encode(
    x="A",
    y="C")

AB | BC | AC

alt.HConcatChart(...)

In [16]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,mAB*mBC,confidence_rAC,confidence_residual_corr,confidence_corrected_bc_corr,confidence_slope_AC,L,U
0,-0.775098,-0.025583,-1.035358,0.650673,1.251633,0.815147,0.999064,0.995636,0.996521,-0.450323,...,-0.001817,-0.990693,-0.000743,0.814403,less,more,less,within,0.994799,0.996338
1,8.904610,-8.787732,-0.878001,0.371220,0.997893,0.385510,0.961491,0.911315,0.948972,-0.639032,...,-0.072751,-0.909534,-0.015072,0.370438,less,more,less,less,0.895160,0.925080
2,0.140165,-5.918405,-5.913346,1.898029,0.568677,1.080490,0.997882,0.979882,0.979846,-0.157736,...,-0.002040,-0.978108,-0.001125,1.079364,within,more,less,within,0.976061,0.983099
3,1.098522,-4.892895,-3.408569,1.672343,1.033678,1.749042,0.989062,0.971070,0.983226,-0.645872,...,-0.022777,-0.970141,-0.020378,1.728664,less,more,less,less,0.965604,0.975678
4,2.706206,6.427357,9.254191,1.108072,1.262204,1.406091,0.993118,0.976919,0.980599,-0.416430,...,-0.010403,-0.974493,-0.007478,1.398613,less,more,less,within,0.972543,0.980605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.787456,12.931904,5.109620,0.069369,0.272050,0.091883,0.248544,0.047338,0.278899,-0.247262,...,-0.267133,-0.047214,-0.073011,0.018872,less,more,within,less,-0.040523,0.134472
96,0.991016,-3.015291,-1.639989,0.468991,1.389019,0.657488,0.990742,0.966148,0.975069,-0.510969,...,-0.017865,-0.965249,-0.006051,0.651437,less,more,less,within,0.959771,0.971529
97,0.067915,1.376628,1.302783,1.918762,0.658953,1.266252,0.998237,0.990301,0.991494,-0.356876,...,-0.002940,-0.990158,-0.001879,1.264374,less,more,less,within,0.988447,0.991858
98,-17.975793,38.158338,10.373910,1.712485,1.485986,2.557455,0.994268,0.989827,0.994022,-0.647894,...,-0.009869,-0.984320,-0.012727,2.544728,less,more,less,within,0.987883,0.991460


In [17]:
linear()

array([314.89480346, 368.0172388 , 416.70247324])